In [1]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver.common.by import By
import pandas as pd
import time
from datetime import datetime
import os

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def catalog_item_to_dict(item):
    data={}
    
    data['name']=item.find_element(By.TAG_NAME, "h4",).get_attribute("innerHTML").replace('<span class="text-fade"></span>', '')
    data['game_number']=item.find_element(By.CLASS_NAME, "game-number").get_attribute("innerHTML").replace("Game #",'')
    data['price']=item.find_element(By.CLASS_NAME, "price").get_attribute("innerHTML").replace('$ ','')
    return data
    

In [3]:
scratchOffUrl = "https://njlottery.com/en-us/scratch-offs/active.html#tab-active"
s=selenium.webdriver.ChromeService()
driver=selenium.webdriver.Chrome(service=s)

driver.get(scratchOffUrl)

items=driver.find_elements(By.CLASS_NAME, "catalog-item")

data=[]
for item in items:
    data.append(catalog_item_to_dict(item))
    
scratchOffData=pd.DataFrame(data)

scratchOffDataCollected = []

for item in scratchOffData.to_dict(orient='records'):
    scratchoff_specific_url = f"https://njlottery.com/content/portal/en/scratch-offs/{item['game_number']}.html"
    driver.get(scratchoff_specific_url)
    
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'prizeChartResult'))
    )

    time.sleep(1)
    htmlData=pd.read_html(driver.find_element(By.ID, "prizeChartResult").get_attribute('innerHTML'))
    htmlData=htmlData[0]
    htmlData['ID'] = item['game_number']
    htmlData["Start"] = driver.find_element(By.CSS_SELECTOR, 'time').get_attribute('innerHTML').strip()
    htmlData['Subtitle'] = driver.find_element(By.CLASS_NAME, 'gameSubTitleDescription').get_attribute('innerHTML').replace("<p>",'').replace("</p>",'')
    htmlData['Title'] = driver.find_elements(By.CLASS_NAME, 'gameTitleDescription')[1].get_attribute('innerHTML').replace("<p>",'').replace("</p>",'')
    scratchOffDataCollected.append(htmlData)

scratchOffs = pd.concat(scratchOffDataCollected)
scratchOff=scratchOffs.merge(scratchOffData, left_on="ID", right_on="game_number", how='right')

scratchOff['DateTimeParsed'] = datetime.now()

_file = os.path.join('data', 'ScratchOffData.csv')
if os.path.exists(_file):
    print(f'Appending {scratchOff.shape[0]} Rows')
    scratchOff.to_csv(_file, index=False, mode='a',header=False)
else:
    print(f'Writing {scratchOff.shape[0]} Rows To New File')
    scratchOff.to_csv(_file, index=False)
driver.close()

C:\Users\Andrew\AppData\Local\Temp\ipykernel_20224\2746482626.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  htmlData=pd.read_html(driver.find_element(By.ID, "prizeChartResult").get_attribute('innerHTML'))
C:\Users\Andrew\AppData\Local\Temp\ipykernel_20224\2746482626.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  htmlData=pd.read_html(driver.find_element(By.ID, "prizeChartResult").get_attribute('innerHTML'))
C:\Users\Andrew\AppData\Local\Temp\ipykernel_20224\2746482626.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  htmlData=pd.read_html(driver.find_element(By.ID, "prizeChartResult").get_attribute('in

Appending 898 Rows


C:\Users\Andrew\AppData\Local\Temp\ipykernel_20224\2746482626.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  htmlData=pd.read_html(driver.find_element(By.ID, "prizeChartResult").get_attribute('innerHTML'))
